In [1]:
"""Simulation of infections for different scenarios."""
import covid19sim.coronalib as cl
import pandas as pd
import numpy as np
import os
import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import plot
from plotly.subplots import make_subplots
import plotly.io as pio


Covid-19 Sim provides two populations:<br>
"current" : The population is based on the current population (2019) <br>
"household" : The population is bases on a subsample in 2010 but with household numbers and additional persons per household

In [2]:
# make a population
age, agegroup, gender, contacts, drate, hnr, persons = cl.makepop("current",
                                                                  17900000)
# set day 0 
day0date = datetime.date(2020, 3, 8)
r_change = {}
# Intial r0
r_change[-1] = 3.3 * contacts/np.mean(contacts)
# First change point (8.3.2020)
r_change[0] = 1.84 * contacts/np.mean(contacts)
# second change point (16.3.2020)
r_change[6] = 1.04 * contacts/np.mean(contacts)
# third change point (23.3.2020)
r_change[13] = 0.9 * contacts/np.mean(contacts)
# fourth change point (20.4.2020)
r_change[42] = 1.1 * contacts/np.mean(contacts)

state, statesum, infections, day0, rnow, args, gr = cl.sim(
        age, drate, nday=150, prob_icu=0.01125,
        day0cumrep=1000,
        mean_days_to_icu=7,mean_duration_icu=10,
        mean_serial=7.0, std_serial=3.0, immunt0=0.0, ifr=0.004,
        long_term_death=False,hnr=None, com_attack_rate=0,
        r_change=r_change, simname="Test", datadir="/mnt/wd1/nrw_corona/",
        realized=None, rep_delay=8.7, alpha=0.125, day0date=day0date)

Simulation time: 123.14643001556396


,index,Peaktag,Peakwert,Peakwert %,Endwert,Endwert %
0,nicht infiziert,-43,17899980,100.000%,17277063,96.520%
1,immun,106,415424,2.321%,415424,2.321%
2,infiziert,106,205675,1.149%,205675,1.149%
3,ICU,104,505,0.003%,490,0.003%
4,tod (Covid-19),106,1348,0.008%,1348,0.008%


In [3]:
fig = make_subplots(rows=4, cols=1)

fig.add_trace(go.Scatter(x=gr["Datum"], y=gr["Erwartete Neu-Meldefälle"],
                        mode="lines",
            name="Erwartete Neu-Meldefälle"),
            row=1, col=1)

fig.add_trace(go.Scatter(x=gr["Datum"], y=gr["Erwartete Gesamt-Meldefälle"],
                        name="Erwartete Gesamt-Meldefälle",
                        mode="lines"), row=2, col=1)

fig.add_trace(go.Scatter(x=gr["Datum"], y=gr["Erwartete Tote"],
                        name="Erwartete Tote",
                        mode="lines"), row=3, col=1)

fig.add_trace(go.Scatter(x=gr["Datum"], y=gr["ICU"],
                        name="Erwartete Intensiv",
                        mode="lines"), row=4, col=1)

fig.update_layout(legend_orientation="h", title=args["simname"])
plot(fig, filename=os.path.join(args["datadir"], args["simname"]+".html"))

'/mnt/wd1/nrw_corona/Test.html'